In [1]:
import json
import spacy
from spacy.matcher import Matcher
from collections import defaultdict

In [2]:
nlp = spacy.load('en_core_web_sm')

c:\Users\sandr\OneDrive\Dokumente\Schule\FFHS\Semester 9\Thesis\GNN-TCG\code\optcg-gnn\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
with open('data/labeled_cards.json') as f:
    cards = json.load(f)

In [ ]:
def parse_card_effect(effect_text):
    """
    Parses the card effect text using NLP to extract structured information.
    """
    doc = nlp(effect_text)

    parsed_effect = {
        'actions': [],
        'conditions': [],
        'targets': []
    }

    # Initialize Matcher
    matcher = Matcher(nlp.vocab)

    # Define patterns for actions
    action_patterns = [
        [{'LEMMA': 'reveal'}],
        [{'LEMMA': 'play'}],
        [{'LEMMA': 'add'}],
        [{'LEMMA': 'draw'}],
        [{'LEMMA': 'return'}],
        [{'LEMMA': 'KO'}],
        [{'LEMMA': 'give'}],
        [{'LEMMA': 'look'}],
        [{'LEMMA': 'trash'}],
        [{'LEMMA': 'rest'}],
        [{'LEMMA': 'gain'}],
        [{'LEMMA': 'set'}],
        [{'LEMMA': 'activate'}],
    ]
    matcher.add('ACTIONS', action_patterns)

    # Define patterns for conditions
    condition_patterns = [
        [{'LOWER': 'if'}, {'OP': '*'}, {'LOWER': 'you'}, {'OP': '*'}, {'LOWER': 'control'}],
        [{'LOWER': 'if'}, {'OP': '*'}, {'LOWER': 'your'}, {'OP': '*'}, {'LOWER': 'leader'}],
        [{'LOWER': 'during'}, {'LOWER': 'your'}, {'LOWER': 'turn'}],
        [{'LOWER': 'once'}, {'LOWER': 'per'}, {'LOWER': 'turn'}],
    ]
    matcher.add('CONDITIONS', condition_patterns)

    # Define patterns for targets
    target_patterns = [
        [{'LOWER': 'character'}, {'LOWER': 'card'}, {'LOWER': 'with'}, {'LOWER': 'a'}, {'LOWER': 'cost'}, {'IS_DIGIT': True}],
        [{'LOWER': 'character'}, {'LOWER': 'card'}, {'LOWER': 'with'}, {'LOWER': 'cost'}, {'IS_DIGIT': True}, {'LOWER': 'or'}, {'LOWER': 'less'}],
        [{'LOWER': 'character'}, {'LOWER': 'card'}, {'LOWER': 'with'}, {'LOWER': 'cost'}, {'IS_DIGIT': True}, {'LOWER': 'to'}, {'IS_DIGIT': True}],
        [{'LOWER': 'character'}, {'LOWER': 'card'}, {'LOWER': 'with'}, {'LOWER': 'trait'}, {'LOWER': '{'}, {'ENT_TYPE': 'TRAIT'}, {'LOWER': '}'}],
        [{'LOWER': 'your'}, {'LOWER': 'opponents'}, {'LOWER': 'character'}, {'LOWER': 'with'}, {'LOWER': 'cost'}, {'IS_DIGIT': True}, {'LOWER': 'or'}, {'LOWER': 'less'}],
    ]
    matcher.add('TARGETS', target_patterns)

    # Apply the matcher to the doc
    matches = matcher(doc)

    # Process matches
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]

        if label == 'ACTIONS':
            action = span.lemma_
            parsed_effect['actions'].append(action)
        elif label == 'CONDITIONS':
            condition = span.text
            parsed_effect['conditions'].append(condition)
        elif label == 'TARGETS':
            target = span.text
            parsed_effect['targets'].append(target)

    # Extract traits and exclusions
    trait_entities = []
    for ent in doc.ents:
        if ent.label_ == 'TRAIT':
            trait_entities.append(ent.text)

    parsed_effect['traits'] = trait_entities

    # Additional parsing logic can be added here

    return parsed_effect

from spacy.language import Language

# Custom NER component to recognize game-specific entities
@Language.component("custom_ner_component")
def custom_ner_component(doc):
    # Add custom entity recognition for traits and other game terms
    for ent in doc.ents:
        continue  # Keep existing entities

    # Manually add entities
    for i, token in enumerate(doc):
        if token.text.startswith('{') and token.text.endswith('}'):
            trait = token.text.strip('{}')
            span = doc[i:i+1]
            doc.ents += ((span.start_char, span.end_char, 'TRAIT'),)

    return doc

# Add custom NER component to the pipeline
nlp.add_pipe("custom_ner_component", after='ner')

def find_synergies(cards):
    """
    Finds synergies between cards based on parsed effects.
    """
    # Parse effects of all cards
    parsed_cards = {}
    for card in cards:
        effect = card.get('effect', '')
        parsed_effect = parse_card_effect(effect)
        card['parsed_effect'] = parsed_effect
        parsed_cards[card['id']] = card

    synergies = defaultdict(list)

    for card in cards:
        card_id = card['id']
        card_name = card['name']
        card_parsed = card['parsed_effect']

        # Initialize synergy list for this card
        synergies[card_id] = {'name': card_name, 'synergies': []}

        # Identify synergies based on actions
        for action in card_parsed['actions']:
            if action == 'reveal':
                # Find trait being searched
                search_traits = card_parsed.get('traits', [])
                exclude_name = card.get('name', '').lower()
                for target_card in cards:
                    if (any(trait in target_card.get('traits', []) for trait in search_traits) and
                            target_card['id'] != card_id and
                            target_card['name'].lower() != exclude_name):
                        synergy_info = {
                            'id': target_card['id'],
                            'name': target_card['name'],
                            'traits': target_card.get('traits', []),
                            'cost': target_card['cost'],
                            'synergy_score': 1
                        }
                        synergies[card_id]['synergies'].append(synergy_info)
            elif action == 'play':
                # Find cards that can be played based on conditions
                targets = card_parsed.get('targets', [])
                for target in targets:
                    # Parse target conditions
                    cost_match = re.search(r'cost (\d+)', target)
                    cost = int(cost_match.group(1)) if cost_match else None
                    if cost:
                        for target_card in cards:
                            if (target_card['type'] == 'Character' and
                                    target_card['cost'] <= cost and
                                    target_card['id'] != card_id):
                                synergy_score = 0
                                if 'on play' in target_card.get('effect', '').lower():
                                    synergy_score += 1
                                if set(card.get('traits', [])) & set(target_card.get('traits', [])):
                                    synergy_score += 1
                                synergy_info = {
                                    'id': target_card['id'],
                                    'name': target_card['name'],
                                    'cost': target_card['cost'],
                                    'synergy_score': synergy_score
                                }
                                synergies[card_id]['synergies'].append(synergy_info)
            elif action == 'add':
                # Handle 'add' actions (e.g., adding from trash)
                conditions = card_parsed.get('conditions', [])
                targets = card_parsed.get('targets', [])
                for target in targets:
                    # Parse target conditions
                    cost_range = re.search(r'cost of (\d+) to (\d+)', target)
                    if cost_range:
                        cost_min = int(cost_range.group(1))
                        cost_max = int(cost_range.group(2))
                        for target_card in cards:
                            if (cost_min <= target_card['cost'] <= cost_max and
                                    target_card['id'] != card_id):
                                synergy_info = {
                                    'id': target_card['id'],
                                    'name': target_card['name'],
                                    'cost': target_card['cost'],
                                    'synergy_score': 1
                                }
                                synergies[card_id]['synergies'].append(synergy_info)

        # Remove duplicates and sort synergies
        unique_synergies = {s['id']: s for s in synergies[card_id]['synergies']}.values()
        synergies[card_id]['synergies'] = sorted(unique_synergies, key=lambda x: x['synergy_score'], reverse=True)

    return synergies



ValueError: [E007] 'custom_ner_component' already exists in pipeline. Existing names: ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner', 'custom_ner_component']

In [ ]:
# print cards head
print(cards[0])

# Run the function and print the synergies
synergy_results = find_synergies(cards)

print (synergy_results)

KeyboardInterrupt: 

In [9]:
for card_id, data in synergy_results.items():
    name = data['name']
    synergies = data['synergies']
    if synergies:
        print(f"Synergies for {name} ({card_id}):")
        for s in synergies:
            print(f" - {s['name']} ({s['id']}), Cost: {s['cost']}, Synergy Score: {s.get('synergy_score', 0)}")
        print()